# 确认milvus使用了余弦相似度

目前默认情况下，milvus的分数值为200左右，明显不在正负1之间。

因此无法通过 similarity_cutoff 过滤。

In [7]:
%%time

import logging
import sys

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai_like import OpenAILike

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

CPU times: user 624 ms, sys: 64.8 ms, total: 689 ms
Wall time: 693 ms


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f4d964e4a90>, id_func=<function default_id_func at 0x7f4e546f71c0>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

llm = OpenAILike(model="qwen14b", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 120 ms, sys: 3.51 ms, total: 124 ms
Wall time: 123 ms


In [4]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 424 ms, sys: 27.5 ms, total: 452 ms
Wall time: 451 ms


In [5]:
%%time

from milvus import default_server
from pymilvus import connections, utility

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

v2.3.5-lite
CPU times: user 3.53 s, sys: 256 ms, total: 3.79 s
Wall time: 7.21 s


In [6]:
%%time

# load documents
documents = SimpleDirectoryReader(input_files=["./books1/故乡.txt"]).load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 68b48806-b570-411f-868c-1c657f97d22e
CPU times: user 9.55 ms, sys: 146 µs, total: 9.69 ms
Wall time: 10.2 ms


In [23]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 412 ms, sys: 15 ms, total: 427 ms
Wall time: 12.2 s


In [30]:
%%time

# Query Data
query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=3,
    similarity_cutoff=300.0
)

CPU times: user 165 µs, sys: 0 ns, total: 165 µs
Wall time: 168 µs


In [31]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是鲁迅小说《故乡》中的人物，他是一个和主角（可能是作者自己）年龄相仿的孩子，因为生于闰月，五行缺土，所以他的父亲给他取名为闰土。他以能装弓捉小鸟雀而闻名。在故事中，闰土来到祭器管理处，并且与主角有过互动。

CPU times: user 103 ms, sys: 16.3 ms, total: 120 ms
Wall time: 3.91 s


In [32]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:10] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

227.4545440673828 故乡.txt 我这时很兴奋，但不知..
218.80593872070312 故乡.txt 闰土来管祭器的。　　..
216.45843505859375 故乡.txt “阿呀，老太太真是…..


## 使用底层api

In [42]:
%%time

from llama_index.core import PromptTemplate

qa_template = (
    "上下文信息如下\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "根据给定的上下文信息回答问题，不要使用任何先验知识。\n"
    "问题: {query_str}\n"
    "回答: "
)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.87 µs


In [46]:
%%time

# Query Data
query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=3,
    # text_qa_template=qa_template
)

CPU times: user 114 µs, sys: 54 µs, total: 168 µs
Wall time: 172 µs


In [47]:
%%time

streaming_response = query_engine.query("孔乙己是谁")
streaming_response.print_response_stream()
print()

从给定的文本中无法找到关于孔乙己的信息。这些文本摘自鲁迅的小说《故乡》，主要描述了主角“我”和少年闰土的故事，以及祭祀等情节，并没有提及孔乙己这个人。因此，根据提供的上下文，无法回答关于孔乙己的问题。




CPU times: user 98.4 ms, sys: 16 ms, total: 114 ms
Wall time: 7.15 s


In [48]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:10] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

158.2081756591797 故乡.txt 闰土来管祭器的。　　..
156.04373168945312 故乡.txt ，反从他的胯下逃走了..
154.89532470703125 故乡.txt “阿呀，老太太真是…..
